In [1]:
import requests
import numpy as np
import pandas as pd
import herepy
# store your herepy API key in a file "config.py" as api_key = "xyz"
import config
import json
import re

In [2]:
# tell the API to use your key
geocoderApi = herepy.GeocoderApi(config.api_key)

# read in data to be geocoded (in this case voter data)
df = pd.read_csv("working2013.csv")

# purge duplicates to minimize geocode API calls
address_df = df.drop_duplicates('address', keep='first')

# make addresses a list
addresses = address_df['address'].unique().tolist()

# remove line breaks within addresses
addresses = [re.sub('\\n',', ',a) for a in addresses]

In [3]:
addresses[:3]

['3955 liberty avenue, pittsburgh, pa',
 '777 casino drive, pittsburgh, pa',
 '1435 bedford avenue, pittsburgh, pa']

In [4]:
# run the geocoder on addresses, currently set to only run on 10 addresses
geocode_responses = []
for i in range(0,len(addresses)):
#for i in range(0,10):
    response = geocoderApi.free_form(addresses[i])
    geocode_responses.append(response.as_dict())

In [5]:
geocode_responses[0]

{'items': [{'title': '3955 Liberty Ave, Pittsburgh, PA 15224-1420, United States',
   'id': 'here:af:streetsection:r882xw.s6Q9XbnR-hBCeWB:CggIBCDaidzFAhABGgQzOTU1KGQ',
   'resultType': 'houseNumber',
   'houseNumberType': 'PA',
   'address': {'label': '3955 Liberty Ave, Pittsburgh, PA 15224-1420, United States',
    'countryCode': 'USA',
    'countryName': 'United States',
    'stateCode': 'PA',
    'state': 'Pennsylvania',
    'county': 'Allegheny',
    'city': 'Pittsburgh',
    'district': 'Lower Lawrenceville',
    'street': 'Liberty Ave',
    'postalCode': '15224-1420',
    'houseNumber': '3955'},
   'position': {'lat': 40.46295, 'lng': -79.95746},
   'access': [{'lat': 40.46281, 'lng': -79.95741}],
   'mapView': {'west': -79.95864,
    'south': 40.46205,
    'east': -79.95628,
    'north': 40.46385},
   'scoring': {'queryScore': 1.0,
    'fieldScore': {'state': 1.0,
     'city': 1.0,
     'streets': [1.0],
     'houseNumber': 1.0}}}]}

In [6]:
# dump these responses to a json file so you won't have to rerun the geocoder api if you need more info from them
with open(r'geocode_2013_responses.json', 'w') as f:
    json.dump(geocode_responses, f)

In [7]:
# code to read it back in, in case you need it later
with open(r"geocode_2013_responses.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
# 

In [8]:
#create blank lists to store results, also store the address as the geocoder recognized it
#in case you need to diagnose errors later
lng_list = []
lat_list = []
recognized_addresses = []
# parse geocode responses. This retrieves
for x in geocode_responses:
    # extract the lat and lon, and append them to the list
    try:
        lng_list.append(x['items'][0]['position']['lng'])
        lat_list.append(x['items'][0]['position']['lat'])
        recognized_addresses.append(x['items'][0]['address']['label'])

    except:
        # if the geocoder did not return a result, just append 0
        lng_list.append(0)
        lat_list.append(0)
        recognized_addresses.append('N/A')

# add these lists as columns to your original dataset
address_df['lon'] = lng_list
address_df['lat'] = lat_list
address_df['geocoder_recognized_address'] = recognized_addresses

# export updated voters file
#test.to_csv("geocoded_2013_data.csv")

<ipython-input-8-090ff72bf510>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_df['lon'] = lng_list
<ipython-input-8-090ff72bf510>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  address_df['lat'] = lat_list
<ipython-input-8-090ff72bf510>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [9]:
address_df

,structure_type,parcel,neighborhood,owner,issued,work description,permit,contractor,address,work_type,...,month,source,mn,split_addr,addr1,addr2,addr3,lon,lat,geocoder_recognized_address
0,not_listed,0049r00244000000,not_listed,john g & valerie s mcconnell,1/15/2013,repairs to south easterly corner of two story ...,11-b-02247,next generation restorable technology,"3955 liberty avenue, pittsburgh, pa",not_listed,...,january,xslx,1,"['3955 liberty avenue', ' pittsburgh', ' pa']",3955 liberty avenue,pittsburgh,pa,-79.95746,40.46295,"3955 Liberty Ave, Pittsburgh, PA 15224-1420, U..."
1,not_listed,0007l00032000000,not_listed,holdings acquisition co lp,1/28/2013,renovate 1st floor offices,12-b-00962,the norwood company,"777 casino drive, pittsburgh, pa",not_listed,...,january,xslx,1,"['777 casino drive', ' pittsburgh', ' pa']",777 casino drive,pittsburgh,pa,-80.02216,40.44720,"777 Casino Dr, Pittsburgh, PA 15212-5840, Unit..."
2,not_listed,0009r00194000000,not_listed,pittsburgh gateways corp,1/22/2013,renovation of former connelly school into the,12-b-01484,mascaro construction company lp,"1435 bedford avenue, pittsburgh, pa",not_listed,...,january,xslx,1,"['1435 bedford avenue', ' pittsburgh', ' pa']",1435 bedford avenue,pittsburgh,pa,-79.99097,40.44300,"1501 Bedford Ave, Pittsburgh, PA 15219, United..."
3,not_listed,0083f00339000000,not_listed,mays thurston & kim (w),1/23/2013,exterior renovations to 2 1/2 story 2 family d...,12-b-02590,marshall builders pa079702,"526 north euclid avenue, pittsburgh, pa",not_listed,...,january,xslx,1,"['526 north euclid avenue', ' pittsburgh', ' pa']",526 north euclid avenue,pittsburgh,pa,-79.92466,40.46745,"526 N Euclid Ave, Pittsburgh, PA 15206-2440, U..."
4,not_listed,0050f00016000000,not_listed,bloomfield-garfield crop,1/23/2013,"new, 2-story single family dwelling with 8' x ...",12-b-02311,catranel steve const,"4914 kincaid street, pittsburgh, pa",not_listed,...,january,xslx,1,"['4914 kincaid street', ' pittsburgh', ' pa']",4914 kincaid street,pittsburgh,pa,-79.94401,40.46731,"4914 Kincaid St, Pittsburgh, PA 15224-1215, Un..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,not_listed,0009r00158000000,not_listed,jakim donaldson,1/29/2013,party wall,13-b-00164,lane plastering and stucco,"814 crawford street, pittsburgh, pa",not_listed,...,january,xslx,1,"['814 crawford street', ' pittsburgh', ' pa']",814 crawford street,pittsburgh,pa,-79.98814,40.44453,"814 Crawford St, Pittsburgh, PA 15219-3622, Un..."
143,not_listed,0049f00130000000,not_listed,costello properties llc,1/29/2013,interior renovations,13-b-00167,costello management llc,"223 fisk street, pittsburgh, pa",not_listed,...,january,xslx,1,"['223 fisk street', ' pittsburgh', ' pa']",223 fisk street,pittsburgh,pa,-79.96060,40.46879,"223 Fisk St, Pittsburgh, PA 15201-2805, United..."
144,not_listed,0052a00022000000,not_listed,hart bill & maria (w),1/29/2013,interior nonstructural demolition in 3 story s...,13-b-00169,buildrx inc,"303 north neville street, pittsburgh, pa",not_listed,...,january,xslx,1,"['303 north neville street', ' pittsburgh', ' ...",303 north neville street,pittsburgh,pa,-79.94938,40.45252,"303 N Neville St, Pittsburgh, PA 15213-1601, U..."
145,not_listed,0012j00143000000,not_listed,leco gerry a & cathy jo (w),1/31/2013,windows and interior renovations to 3 story two,13-b-00183,craig sweitzer construction,"1725 larkins way, pittsburgh, pa",not_listed,...,january,xslx,1,"['1725 larkins way', ' pittsburgh', ' pa']",1725 larkins way,pittsburgh,pa,-79.98111,40.42755,"1725 Larkins Way, Pittsburgh, PA 15203-1815, U..."


In [10]:
# export updated voters file
#voters.to_csv("geocoded_voter_data.csv")

# export updated voters file
address_df.to_csv("geocoded_2013_data.csv")